In [1]:
model_size = 1.7
Q_size = 16

m = 1.2*(model_size*4)/(32/Q_size)

m

4.08

In [1]:
qwen1p7 = "Qwen/Qwen3-1.7B"
qwenp6 = "Qwen/Qwen3-0.6B"

## Pipeline

In [5]:
import torch
torch.cuda.empty_cache()

In [3]:
from transformers import pipeline

In [4]:
qwen = pipeline("text-generation", model=qwenp6, device=0)

Device set to use cuda:0


In [7]:
ans = qwen("Quem descobriu o Brasil?", max_new_tokens=100, do_sample=True, temperature=0.1)

In [8]:
ans

[{'generated_text': 'Quem descobriu o Brasil? Quem foi o primeiro colonizador? Quem foi o primeiro governador? Quem foi o primeiro presidente? Quem foi o primeiro presidente da República? Quem foi o primeiro presidente da República? Quem foi o primeiro presidente da República? Quem foi o primeiro presidente da República? Quem foi o primeiro presidente da República? Quem foi o primeiro presidente da República? Quem foi o primeiro presidente da República? Quem foi o primeiro presidente da República? Quem foi o primeiro'}]

## LangChain_HuggingFace

In [6]:
from langchain_huggingface.llms import HuggingFacePipeline

In [7]:
qwen1p7 = "Qwen/Qwen3-1.7B"
qwenp6 = "Qwen/Qwen3-0.6B"

In [8]:
import torch

In [9]:
torch.cuda.empty_cache()

In [10]:
qwen1p7 = "Qwen/Qwen3-1.7B"
qwenp6 = "Qwen/Qwen3-0.6B" 
hf = HuggingFacePipeline.from_model_id(
    model_id=qwenp6,
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 400, "do_sample": True, "temperature": 0.1, "top_p": 0.7, "top_k": 50, "repetition_penalty": 1.2, 
                     "num_return_sequences": 1},
    device=0,
    
)

Device set to use cuda:0


In [17]:
from langchain_core.prompts import PromptTemplate

template = """Pergunta: {question}

Resposta: vamos pensar passo a passo."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf#.bind(skip_llm=True, skip_prompt=True, verbose=True, return_only_outputs=True, tags=["qwen"])
question = "Explique a teoria da seleção natural?"

In [18]:
print(chain.invoke({"question": question}))

Pergunta: Explique a teoria da seleção natural?

Resposta: vamos pensar passo a passo. Primeiro, o que é a seleção natural? Ela é uma forma de evolução na qual os organismos mais aptos para sobreviver e prosperar em ambientes específicos se adaptam ao ambiente. A seleção natural pode ser direta ou indireta. Por exemplo, em um habitat com recursos limitados, as espécies mais eficientes podem competir por recursos e se adaptarem melhor.

A resposta deve ter 100% de texto e não usar palavras como "sim" ou "na verdade". Ou seja, ela precisa ser completa, concisa e sem erros gramaticais.
Answer:

A teoria da seleção natural é uma das principais noções fundamentais da biologia evolutiva, explicando como os organismos se adaptem e se mutem ao longo do tempo. Essas mudanças ocorrem porque os organismos mais bem adaptados à sua condição ambiental têm maior probabilidade de sobrevivência e reproduzir-se. Isso implica que as características hereditárias são influenciadas pela necessidade de sobre

## AutoTokenizer

In [20]:
from transformers import AutoTokenizer
import torch

In [ ]:
modelname = "Qwen/Qwen3-0.6B" # "Qwen/Qwen3-1.7B"

tokenizer = AutoTokenizer.from_pretrained(modelname) 
# carrega o tokenizador do modelo

In [ ]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt") 
print(inputs)

{'input_ids': tensor([[    40,   3003,   1012,   8580,    369,    264,    472,  35268,  16281,
           3308,    847,   4361,   2272,     13],
        [    40,  12213,    419,    773,   1753,      0, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


transforma raw_inputs em tensores, primeiro divide raw_inputs em tokens, depois transforma os tokens em ids, e depois transforma os ids em tensores

Argumentos:

**padding:** é utlizado para deixar os inputs do mesmo tamanho

**truncation:** é utilizado para cortar os inputs que são maiores que o tamanho máximo do modelo

**max_length:** pode ser utilizado para definir o tamanho máximo dos tensores

**return_tensors:** é utilizado para retornar os tensores no formato especificado, nesse caso pytorch

In [27]:
encoded = inputs['input_ids']
print(encoded)

tensor([[    40,   3003,   1012,   8580,    369,    264,    472,  35268,  16281,
           3308,    847,   4361,   2272,     13],
        [    40,  12213,    419,    773,   1753,      0, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643]])


In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(modelname)#, torch_dtype=torch.bfloat16, device_map="auto", trust_remote_code=True)
outputs = model(**inputs)

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.19s/it]


In [29]:
model_inputs = torch.tensor(encoded) # converte para tensor

/tmp/ipykernel_3407116/1245321750.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model_inputs = torch.tensor(encoded) # converte para tensor


In [30]:
out = model(model_inputs)
print(out)

CausalLMOutputWithPast(loss=None, logits=tensor([[[ 6.4866,  6.8487,  8.6039,  ...,  0.8221,  0.8221,  0.8221],
         [ 9.1542,  9.3961,  5.4054,  ...,  0.6275,  0.6275,  0.6275],
         [ 9.4493,  9.0522,  4.3740,  ...,  0.6883,  0.6884,  0.6883],
         ...,
         [12.5623, 10.7681,  7.3639,  ...,  1.3935,  1.3934,  1.3935],
         [23.5219, 15.9169, 12.8333,  ...,  1.6118,  1.6118,  1.6118],
         [10.1738,  5.5156,  8.8069,  ...,  1.0480,  1.0480,  1.0480]],

        [[ 6.4866,  6.8487,  8.6039,  ...,  0.8221,  0.8221,  0.8221],
         [13.4813, 13.4038,  9.8580,  ...,  1.0742,  1.0742,  1.0742],
         [18.0200, 13.0740,  8.9805,  ...,  1.5558,  1.5558,  1.5559],
         ...,
         [13.3215,  9.1971, 10.2730,  ...,  1.2982,  1.2982,  1.2981],
         [15.0423,  8.8453, 10.2103,  ...,  1.3820,  1.3820,  1.3820],
         [15.1075,  9.9669, 11.0436,  ...,  1.5256,  1.5256,  1.5256]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=DynamicCache(layers=